# JEM092 Asset Pricing
# Seminar 8
## Lukáš Petrásek
### Charles University
### lukas.petrasek@fsv.cuni.cz
## 14.4.2022

In [ ]:
# Import third-party packages.
library(PerformanceAnalytics)
library(portsort)
library(quantmod)
library(readr)
library(sandwich)
library(xts)

# Data preparation

In [ ]:
# Load tickers.
sp100_tickers <- read_csv(
    '/home/lukas/projects/asset-pricing/summer-semester-2022/seminar_8/sp100_tickers.csv',
    col_names = FALSE,
)

In [ ]:
START_DATE <- '2011-01-01'
END_DATE <- '2022-03-31'

# Load stock prices and nominals.
prices <- c()
nominals <- c()
for (i in 1:nrow(sp100_tickers)) {
    ticker <- as.character(sp100_tickers[i, 1])
    stock_data <- getSymbols(
        ticker,
        from = START_DATE,
        to = END_DATE,
        src = 'yahoo',
        warnings = FALSE,
        auto.assign = FALSE
    )

    stock_prices = stock_data[, 6]
    stock_nominals = stock_data[, 5] * stock_prices
    colnames(stock_prices) <- ticker
    colnames(stock_nominals) <- ticker
    prices <- cbind(prices, stock_prices)
    nominals <- cbind(nominals, stock_nominals)
}

In [ ]:
prices[1:5, 1:3]
nominals[1:5, 1:3]

In [ ]:
# Compute daily and quarterly returns.
daily_returns <- lapply(prices, dailyReturn, USE.NAMES = TRUE)
quarterly_returns <- lapply(prices, quarterlyReturn, USE.NAMES = TRUE)
daily_returns <- do.call('cbind', daily_returns)
quarterly_returns <- do.call('cbind', quarterly_returns)
colnames(daily_returns) <- paste('dr', colnames(prices), sep = '_')
colnames(quarterly_returns) <- paste('qr', colnames(prices), sep = '_')
index(quarterly_returns) <- as.Date(as.yearqtr(index(quarterly_returns), format = "Q%q/%y"), frac = 1)

# Compute quarterly volatilities.
quarterly_volatilities <- as.xts(aggregate(daily_returns, as.yearqtr(as.yearmon(time(daily_returns))), sd))
index(quarterly_volatilities) <- as.Date(as.yearqtr(index(quarterly_volatilities), format = "Q%q/%y"), frac = 1)
colnames(quarterly_volatilities) <- paste('qv', colnames(prices), sep = '_')

# Comupte quarterly nominals.
quarterly_nominals <- as.xts(aggregate(nominals, as.yearqtr(as.yearmon(time(nominals))), sum))
index(quarterly_nominals) <- as.Date(as.yearqtr(index(quarterly_nominals), format = "Q%q/%y"), frac = 1)
colnames(quarterly_nominals) <- paste('qn', colnames(prices), sep = '_')

In [ ]:
# Combine the quarterly data.
head(cbind(quarterly_returns, quarterly_volatilities, quarterly_nominals)[, 96:100])
head(cbind(quarterly_returns, quarterly_volatilities, quarterly_nominals)[, 193:197])

# Portfolio sorts

Sorting procedure:

1) Find breakpoints.

2) Form portfolios.

3) Average values within each portfolio.

## Univariate sorts

Sorts are created based on a single variable.

### Example: 3 months quintile portfolios based on nominal

In [ ]:
n_portfolios <- 5
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns_nominals <- as.xts(
    matrix(
        nrow = nrow(quarterly_nominals) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(quarterly_nominals[2:nrow(quarterly_nominals)])
)
names(returns_nominals) <- c(1:n_portfolios, diff_portfolio)

# Let's fix `i` so that we easily see what happens inside the first loop.
i <- 3

# Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
quarter_nominals <- quarterly_nominals[i]
quarter_returns <- quarterly_returns[i + 1]

breakpoints <- quantile(quarter_nominals, 0:n_portfolios/n_portfolios, na.rm = TRUE)
not_na <- !is.na(quarter_nominals)

for (j in 1:n_portfolios) {
    filter <- (breakpoints[[j]] < quarter_nominals) & (quarter_nominals < breakpoints[[j + 1]]) & not_na
    returns_nominals[i, j] <- mean(quarter_returns[, filter])
}
returns_nominals[i, diff_portfolio] <- returns_nominals[i, n_portfolios] - returns_nominals[i, 1]

head(returns_nominals)

In [ ]:
n_portfolios <- 5
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns_nominals <- as.xts(
    matrix(
        nrow = nrow(quarterly_nominals) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(quarterly_nominals[2:nrow(quarterly_nominals)])
)
names(returns_nominals) <- c(1:n_portfolios, diff_portfolio)

# Iterate over rows, find breakpoints and compute quarterly returns within the given nominal bounds.
for (i in 1:nrow(returns_nominals)) {
    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    quarter_nominals <- quarterly_nominals[i]
    quarter_returns <- quarterly_returns[i + 1]

    breakpoints <- quantile(quarter_nominals, 0:n_portfolios/n_portfolios, na.rm = TRUE)
    not_na <- !is.na(quarter_nominals)

    for (j in 1:n_portfolios) {
        filter <- (breakpoints[[j]] < quarter_nominals) & (quarter_nominals < breakpoints[[j + 1]]) & not_na
        returns_nominals[i, j] <- mean(quarter_returns[, filter])
    }
    returns_nominals[i, diff_portfolio] <- returns_nominals[i, n_portfolios] - returns_nominals[i, 1]
}

# Compute overall average returns within portfolios and their standard errors.
results_nominals <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_nominals) <- c(1:n_portfolios, diff_portfolio)
for (i in 1:ncol(results_nominals)) {
    model <- lm(returns_nominals[, i] ~ 1)
    results_nominals[1, i] <- model$coefficients[[1]]
    results_nominals[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

results_nominals

### Example: 3 months quintile portfolios based on volatility

In [ ]:
# Choose the number of portfolios.
n_portfolios <- 5
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns_volatilities <- as.xts(
    matrix(
        nrow = nrow(quarterly_volatilities) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(quarterly_volatilities[2:nrow(quarterly_volatilities)])
)
names(returns_volatilities) <- c(1:n_portfolios, diff_portfolio)

# Iterate over rows, find breakpoints and compute quarterly returns within the given volatility bounds.
for (i in 1:nrow(returns_volatilities)) {
    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    quarter_volatilities <- quarterly_volatilities[i]
    quarter_returns <- quarterly_returns[i + 1]

    breakpoints <- quantile(quarter_volatilities, 0:n_portfolios/n_portfolios, na.rm = TRUE)
    not_na <-  !is.na(quarter_volatilities)

    for (j in 1:n_portfolios) {
        filter <- (breakpoints[[j]] < quarter_volatilities) & (quarter_volatilities < breakpoints[[j + 1]]) & not_na
        returns_volatilities[i, j] <- mean(quarter_returns[, filter])
    }
    returns_volatilities[i, diff_portfolio] <- returns_volatilities[i, n_portfolios] - returns_volatilities[i, 1]
}

# Compute overall average returns within portfolios and their standard errors.
results_volatilities <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_volatilities) <- c(1:n_portfolios, diff_portfolio)
for (i in 1:ncol(results_volatilities)) {
    model <- lm(returns_volatilities[, i] ~ 1)
    results_volatilities[1, i] <- model$coefficients[[1]]
    results_volatilities[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

results_volatilities

## Bivariate sorts

Sorts are created based on a two variables.

### Example: 4 * 3 portfolios based on returns and volumes

Let's follow the example in the documentation of the `portsort` package.

#### Independent sorts

Breakpoints of both variables are independent (computed on all observations).

In [ ]:
# Firstly, load the pre-loaded data.
data(Factors)

# Leading returns, lagged returns, and lagged volumes are stored in the Factors list.
R.Forward <- Factors[[1]]
R.Lag <- Factors[[2]]
V.Lag <- Factors[[3]]
Fa <- R.Lag
Fb <- V.Lag

# Specify the dimension of the sort - let's use quartiles and terciles.
dimA <- 0:4/4
dimB <- 0:3/3
dimC <- c(0,1)

# Run the unconditional sort with quantiles computed using method 7 from the quantile function (stats package).
sort.output.uncon <- unconditional.sort(Fa, Fb, Fc = NULL, R.Forward, dimA, dimB, dimC, type = 7)

# Compare the risk and return of each sub-portfolio using PerformanceAnalytics.
# Set the scale to 365 (Cryptocurrency markets have no close) and geometric to FALSE (we are using log returns).
table.AnnualizedReturns(sort.output.uncon$returns, scale = 365, geometric = FALSE, digits = 3)

# Following the methodology of Gargano et al. 2017 and Bianchi and Dickerson (2018), we will now form a long-short, 
# zero-cost portfolio which initiates a long position in the low prior return/low volume sub-portfolio (sub-portfolio
# 1) and a short position in the low return/high volume sub-portfolio (sub-portfolio 9).
ls_portfolio.uncon = sort.output.uncon$returns[, 1] + (-1 * sort.output.uncon$returns[, 9])
colnames(ls_portfolio.uncon) = c('Unonditional')

# Plot the logarithmic cumulative returns.
chart.CumReturns(ls_portfolio.uncon, geometric = FALSE, legend.loc = 'topleft')

# Investigate risk and return.
table.AnnualizedReturns(ls_portfolio.uncon, scale = 365, geometric = FALSE)

#### Dependent sorts

Breakpoints of the second sort variable are computed on observations within each group formed based on the first sort variable.

In [ ]:
# Run the conditional sort with quantiles computed using method 7 from the quantile function (stats package).
sort.output.con <- conditional.sort(Fa, Fb, Fc = NULL, R.Forward, dimA, dimB, dimC, type = 7)

# Compare the risk and return of each sub-portfolio using PerformanceAnalytics.
# Set the scale to 365 (Cryptocurreny markets have no close) and geometric to FALSE (we are using log returns).
table.AnnualizedReturns(sort.output.con$returns, scale = 365, geometric = FALSE, digits = 3)

# Following the methodology of Gargano et al. 2017 and Bianchi and Dickerson (2018), we will now form a long-short, 
# zero-cost portfolio which initiates a long position in the low prior return/low volume sub-portfolio (sub-portfolio
# 1) and a short position in the low return/high volume sub-portfolio (sub-portfolio 9).
ls_portfolio.con = sort.output.con$returns[, 1] + (-1 * sort.output.con$returns[, 9])
colnames(ls_portfolio.con) = c('Conditional')

# Plot the logarithmic cumulative returns.
chart.CumReturns(ls_portfolio.con, geometric = FALSE, legend.loc = 'topleft')

# Investigate risk and return.
table.AnnualizedReturns(ls_portfolio.con, scale = 365, geometric = FALSE)

#### Compare the sorts

In [ ]:
portfolios = cbind(ls_portfolio.con, ls_portfolio.uncon)
colnames(portfolios) = c('Conditional', 'Unconditional')

# Plot the logarithmic cumulative returns.
chart.CumReturns(portfolios, geometric = FALSE, legend.loc = 'topleft', main = 'Sorting Comparison')

# Investigate risk and return.
table.AnnualizedReturns(portfolios,scale = 365, geometric = FALSE)

# Next time: Fama-MacBeth regressions

Quoting from the fama-macbeth package documentation:

Theories of asset pricing frequently use 'risk factors' to explain asset returns. These factors can range from macroeconomic (for example, consumer inflation or the unemployment rate) to financial (firm size, etc). The Fama-MacBeth two-step regression is a practical way of testing how these factors describe portfolio or asset returns. The goal is to find the premium from exposure to these factors. In the first step, each portfolio's return is regressed against one or more factor time series to determine how exposed it is to each one (the 'factor exposures'). In the second step, the cross-section of portfolio returns is regressed against the factor exposures, at each time step, to give a time series of risk premia coefficients for each factor. The insight of Fama-MacBeth is to then average these coefficients, once for each factor, to give the premium expected for a unit exposure to each risk factor over time.

In equation form, for $n$ portfolio or asset returns and $m$ factors, in the first step the factor exposure $\beta$s are obtained by calculating $n$ regressions, each one on $m$ factors (each equation in the following represents a regression):
$$R_{1,t} = \alpha_1 + \beta_{1,F_1} F_{1,t} + \beta_{1,F_2} F_{2,t} + . . . + \beta_{1,F_m} F_{m,t} + \epsilon_{1,t}$$
$$R_{2,t} = \alpha_2 + \beta_{2,F_1} F_{1,t} + \beta_{2,F_2} F_{2,t} + . . . + \beta_{2,F_m} F_{m,t} + \epsilon_{2,t}$$
.
.
.
$$R_{n,t} = \alpha_n + \beta_{n,F_1} F_{1,t} + \beta_{n,F_2} F_{2,t} + . . . + \beta_{n,F_m} F_{m,t} + \epsilon_{n,t}$$
where $R_{i,t}$ is the return of portfolio or asset $i$ ($n$ total) at time $t$, $F_{j,t}$ is the factor $j$ ($m$ total) at time $t$, $\beta_{i,F_m}$ are the factor exposures, or loadings, that describe how returns are exposed to the factors, and $t$ goes from $1$ through $T$. Notice that each regression uses the same factors $F$, because the purpose is to determine the exposure of each portfolio's return to a given set of factors.

The second step is to compute $T$ cross-sectional regressions of the returns on the $m$ estimates of the $\beta$s (call then $\hat{\beta}$) calculated from the first step. Notice that each regression uses the same $\beta$s from the first step, because now the goal is the exposure of the $n$ returns to the $m$ factor loadings over time (e.g., does a larger factor exposure mean a higher return?):
$$R_{i,1} = \gamma_{1,0} + \gamma_{1,1} \hat{\beta}_{i,F_1} + + \gamma_{1,2} \hat{\beta}_{i,F_2} + . . . + + \gamma_{1,m} \hat{\beta}_{i,F_m} + \epsilon_{i,1}$$
$$R_{i,2} = \gamma_{2,0} + \gamma_{2,1} \hat{\beta}_{i,F_1} + + \gamma_{2,2} \hat{\beta}_{i,F_2} + . . . + + \gamma_{2,m} \hat{\beta}_{i,F_m} + \epsilon_{i,2}$$
.
.
.
$$R_{i,T} = \gamma_{T,0} + \gamma_{1,1} \hat{\beta}_{i,F_1} + + \gamma_{T,2} \hat{\beta}_{i,F_2} + . . . + + \gamma_{T,m} \hat{\beta}_{i,F_m} + \epsilon_{i,T}$$
where the returns $R$ are the same as those in the first step equations, $\gamma$ are regression coefficients that are later used to calculate the risk premium for each factor, and in each regression $i$ goes from $1$ through $n$.

In the end there are $m + 1$ series $\gamma$ (including the constant in the second step) for every factor, each of length $T$. If the $\epsilon$ are assumed to be i.i.d, calculate the risk premium $\gamma_m$ for factor $F_m$ by averaging the $m$th $\gamma$ over $T$, and also get standard deviations and t-stats. For example, t-stats for the mth risk premium are:
$$\frac{\gamma_m}{\sigma_{\gamma_m} / \sqrt{T}}$$